In [198]:
import pandas as pd
import numpy as np

In [199]:
num_questions = 10

In [200]:
df = pd.read_csv('data/data.csv', sep="\t")

In [201]:
factors = {'EXT': 'extraversion', 'EST': 'stability', 'AGR': 'agreeableness', 'CSN': 'conscientiousness', 'OPN': 'openness'}
inversion = {'EXT': {2, 4, 6, 8, 10}, 'AGR': {1, 3, 5, 7}, 'EST': {1, 3, 5, 6, 7, 8, 9, 10}, 'CSN': {2, 4, 6, 8}, 'OPN': {2, 4, 6}}
question_cols = []
question_speed_cols = []
question_summary_cols = factors.values()

In [202]:
for fact in factors.keys():
    newdf = pd.DataFrame()
    s = np.zeros(df.shape[0])
    for i in range(1, num_questions+1):
        newdf[f'{factors[fact]}{i}'] = df[f'{fact}{i}']
        if i in inversion[fact]:
            newdf[f'{factors[fact]}{i}'] = 6 - newdf[f'{factors[fact]}{i}']
        s += newdf[f'{factors[fact]}{i}']
        newdf[f'{factors[fact]}{i}_E'] = df[f'{fact}{i}_E']
        df = df.drop([f'{fact}{i}', f'{fact}{i}_E'], axis=1)
        question_cols.append(f'{factors[fact]}{i}')
        question_speed_cols.append(f'{factors[fact]}{i}_E')
    newdf[factors[fact]] = s / num_questions
    df = pd.concat([newdf, df], axis=1)

In [203]:
df = df.copy() # defragment

In [204]:
df.country = df.country.astype('category')

In [205]:
df['startTime'] = pd.to_datetime(df.dateload)
df = df.drop(['dateload'], axis=1)

In [206]:
df.lat_appx_lots_of_err = df.lat_appx_lots_of_err.replace("NONE", np.nan).astype('float')
df.long_appx_lots_of_err = df.long_appx_lots_of_err.replace("NONE", np.nan).astype('float')

In [207]:
demo_cols = df.columns.difference(question_cols).difference(question_speed_cols).difference(question_summary_cols)

In [208]:
df.endelapse = df.endelapse.astype('int')

In [209]:
df = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015341 entries, 0 to 1015340
Columns: 115 entries, openness1 to startTime
dtypes: category(1), datetime64[ns](1), float64(111), int64(2)
memory usage: 885.0 MB


In [210]:
df.to_hdf('data/data.h5', key='all', format='table')

In [211]:
df = df.dropna(subset=question_cols)
for q in question_cols:
    df[q] = df[q].astype('int')
for q in question_speed_cols:
    df[q] = df[q].astype('int')

In [212]:
df.to_hdf('data/data.h5', key='all_questions', format='table')